# Model to Predict Water Potability - 1st part

## Data cleaning

In [1]:
import pandas as pd
import numpy as np

In [2]:
water_df = pd.read_csv("data/water_potability.csv")
water_df.head()

ph    Hardness        Solids  Chloramines     Sulfate  Conductivity  \
0       NaN  204.890455  20791.318981     7.300212  368.516441    564.308654   
1  3.716080  129.422921  18630.057858     6.635246         NaN    592.885359   
2  8.099124  224.236259  19909.541732     9.275884         NaN    418.606213   
3  8.316766  214.373394  22018.417441     8.059332  356.886136    363.266516   
4  9.092223  181.101509  17978.986339     6.546600  310.135738    398.410813   

   Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       10.379783        86.990970   2.963135           0  
1       15.180013        56.329076   4.500656           0  
2       16.868637        66.420093   3.055934           0  
3       18.436524       100.341674   4.628771           0  
4       11.558279        31.997993   4.075075           0

Displaying the first five rows, we can see that we have missing values, which we have to handle before training the model.

In [3]:
print(f"Dataset shape: {water_df.shape}")
print("\nFeatures with missing values: ")
print(water_df.isnull().sum())

Dataset shape: (3276, 10)

Features with missing values: 
ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64


In [4]:
print(f"Shape after drop all missing values: {water_df.dropna().shape}")

Shape after drop all missing values: (2011, 10)


If we drop all the missing values from the data frame, we lose more than 30% of the dataset. Instead, we will fill this data using statistics values. But first, we will see an entire dataset description.

In [5]:
water_df.describe()

ph     Hardness        Solids  Chloramines      Sulfate  \
count  2785.000000  3276.000000   3276.000000  3276.000000  2495.000000   
mean      7.080795   196.369496  22014.092526     7.122277   333.775777   
std       1.594320    32.879761   8768.570828     1.583085    41.416840   
min       0.000000    47.432000    320.942611     0.352000   129.000000   
25%       6.093092   176.850538  15666.690297     6.127421   307.699498   
50%       7.036752   196.967627  20927.833607     7.130299   333.073546   
75%       8.062066   216.667456  27332.762127     8.114887   359.950170   
max      14.000000   323.124000  61227.196008    13.127000   481.030642   

       Conductivity  Organic_carbon  Trihalomethanes    Turbidity   Potability  
count   3276.000000     3276.000000      3114.000000  3276.000000  3276.000000  
mean     426.205111       14.284970        66.396293     3.966786     0.390110  
std       80.824064        3.308162        16.175008     0.780382     0.487849  
min      181.483754        2.200000         0.738000     1.450000     0.000000  
25%      365.734414       12.065801        55.844536     3.439711     0.000000  
50%      421.884968       14.218338        66.622485     3.955028     0.000000  
75%      481.792304       16.557652        77.337473     4.500320     1.000000  
max      753.342620       28.300000       124.000000     6.739000     1.000000

In [6]:
# First with Trihalomethanes feature, which has the less missing values
triahalomethanes_median = water_df["Trihalomethanes"].median()
water_df["Trihalomethanes"] = water_df["Trihalomethanes"].fillna(triahalomethanes_median)
print(water_df.isnull().sum())
print(f"\nShape after fill Trihalomethanes feature and drop all missing values: {water_df.dropna().shape}")

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes      0
Turbidity            0
Potability           0
dtype: int64

Shape after fill Trihalomethanes feature and drop all missing values: (2116, 10)


pH and Sulfate features has a lot of missing values, fill it with the mediam may cause some problems when we train. This features will be filled with pandas _ffill_ method. 

In [8]:
water_df["ph"] = water_df["ph"].fillna(method="bfill") # For pH feature
water_df["Sulfate"] = water_df["Sulfate"].fillna(method="ffill") # Sulfate feature
print("Missing values after fill pH and Sulfate features: \n", water_df.isnull().sum())

Missing values after fill pH and Sulfate features: 
 ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64


C:\Users\Banapuram\AppData\Local\Temp\ipykernel_24620\773889011.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  water_df["ph"] = water_df["ph"].fillna(method="bfill") # For pH feature
C:\Users\Banapuram\AppData\Local\Temp\ipykernel_24620\773889011.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  water_df["Sulfate"] = water_df["Sulfate"].fillna(method="ffill") # Sulfate feature


In [9]:
water_df.describe()

ph     Hardness        Solids  Chloramines      Sulfate  \
count  3276.000000  3276.000000   3276.000000  3276.000000  3276.000000   
mean      7.068424   196.369496  22014.092526     7.122277   333.946231   
std       1.599381    32.879761   8768.570828     1.583085    41.261830   
min       0.000000    47.432000    320.942611     0.352000   129.000000   
25%       6.087306   176.850538  15666.690297     6.127421   307.652451   
50%       7.037422   196.967627  20927.833607     7.130299   333.076588   
75%       8.062251   216.667456  27332.762127     8.114887   360.173794   
max      14.000000   323.124000  61227.196008    13.127000   481.030642   

       Conductivity  Organic_carbon  Trihalomethanes    Turbidity   Potability  
count   3276.000000     3276.000000      3276.000000  3276.000000  3276.000000  
mean     426.205111       14.284970        66.407478     3.966786     0.390110  
std       80.824064        3.308162        15.769958     0.780382     0.487849  
min      181.483754        2.200000         0.738000     1.450000     0.000000  
25%      365.734414       12.065801        56.647656     3.439711     0.000000  
50%      421.884968       14.218338        66.622485     3.955028     0.000000  
75%      481.792304       16.557652        76.666609     4.500320     1.000000  
max      753.342620       28.300000       124.000000     6.739000     1.000000

We didn't alter our stats, as we see in the new description. We can continue to EDA.

In [10]:
water_df.to_csv("data/clean_dataset.csv", index=False)